---
title : "Belastingen inladen"
excecute:
    output: asis
---

Inlezen waterstanden uit verschillende bronsystemen:
- RWS Waterinfo
- FEWS
- (Noos) Matroos
- RWS WaterWebServices

Gebruikmakend van:
- csv bestanden voor uitwisselen van data

### Benodigde generieke Python packages

In [ ]:
from pathlib import Path

### Definitie configuratie: CSV bestanden 

GlobalVariables:
    rootdir: "data_demo" 
    moments: [-24,0,24,48]  

    LoadsWaterinfo:
        parameters: ["waterhoogte"]
        MISSING_VALUE: 999

    LoadsFews:
        host: "https://fews.hhnk.nl"
        port: 443
        region: "fewspiservice"
        version: "1.25"
        filter: "HHNK_WEB"
        parameters: ["WATHTE [m][NAP][OW]", "WINDRTG [deg]"]
        aquo_equivalent: ["WATHTE","WINDRTG"]
        MISSING_VALUE: 999
    
    LoadsMatroos:
        website: "vitaal"
        model: "dcsm6_kf"
        parameters: ["WATHTE"]
        MISSING_VALUE: 999
    
    LoadsWaterwebservicesRWS:
        parameters: ["WATHTE"]
        MISSING_VALUE: 999

DataAdapter: 
    default_options:
        csv_source:
            sep: ";"
        csv:
            sep: ","
    BelastingLocaties_WaterInfo: 
        type: csv_source
        filter: "WaterInfo"
        path: "locaties.csv"
    Waterstanden_WaterInfo: 
        type: csv
        path: "waterstanden_waterinfo.csv"
    BelastingLocaties_FEWS: 
        type: csv_source
        filter: "FEWS"
        path: "locaties.csv"
    Waterstanden_FEWS: 
        type: csv
        path: "waterstanden_fews.csv"
    BelastingLocaties_Matroos: 
        type: csv_source
        filter: "NOOS Matroos"
        path: "locaties.csv"
    Waterstanden_Matroos: 
        type: csv
        path: "waterstanden_matroos.csv"
    BelastingLocaties_Waterwebservices_RWS: 
        type: csv_source
        filter: "RWS Waterwebservices"
        path: "locaties.csv"
    Waterstanden_Waterwebservices_RWS: 
        type: csv
        path: "waterstanden_waterwebservices_rws.csv"

### Lees configuratie in en maak dictionary base.Config aan

In [ ]:
from toolbox_continu_inzicht import Config

yaml_config_file = "demo_sprint2-I.yaml"
path = Path.cwd() / "data_demo" / yaml_config_file
c = Config(config_path=path)
c.lees_config()

### Geef config mee aan base.Adapter

In [ ]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)
print(f"Folder met data: {data.config.config_path}")
print(f"Globale variabelen: {data.config.global_variables}")
for data_adapter in data.config.data_adapters:
    print(f"Data apdater: {data_adapter}")

### Invoerbestand (CSV) met data van belastinglocaties
Verplichte velden (schema):
- measurement_location_id: int64
- measurement_location_code: object (string)
- measurement_location_description: object (string)

measurement_location_id;measurement_location_code;measurement_location_description;source;tide\
1;Pannerdense-kop(PANN);Pannerdense kop;WaterInfo;false\
2;MPN-N-24;Meetpunt Spijkerboor;FEWS;false\
3;hoekvanholland;Hoek van Holland;NOOS Matroos;true\
4;9889;Hintham beneden;RWS Waterwebservices;false

### Laad module (functie) LoadsWaterinfo voor inlezen belastingen uit RWS Waterinfo

In [ ]:
from toolbox_continu_inzicht.loads import LoadsWaterinfo

waterinfo = LoadsWaterinfo(data_adapter=data)

### Run module (functie) LoadsWaterinfo voor inlezen waterstanden uit Waterinfo
LET OP: zoek op basis van een measurement_location_code die in RWS WaterInfo overeenkomt met ['locationCodes'](https://waterinfo.rws.nl/api/chart/get?mapType=waterhoogte&locationCodes=Pannerdense-kop(PANN)&values=-48%2C48)

In [ ]:
ds = waterinfo.run(input="BelastingLocaties_WaterInfo", output="Waterstanden_WaterInfo")

In [ ]:
measuringlocations = waterinfo.df_in
measuringlocations

In [ ]:
waterlevels = waterinfo.df_out
waterlevels

### Laad en run module (functie) LoadsFEWS voor inlezen belastingen uit FEWS
LET OP: zoek op basis van een measurement_location_code die in FEWS overeenkomt met ['locationId'](https://fews.hhnk.nl/FewsWebServices/rest/fewspiservice/v1/locations)<br>
![image info](data_demo/images/FEWS_locationIs.PNG)

FEWS is niet extern bereikbaar

In [ ]:
# from toolbox_continu_inzicht.loads import LoadsFews

# fews = LoadsFews(data_adapter=data)
# ds = fews.run(input="BelastingLocaties_FEWS", output="Waterstanden_FEWS")

In [ ]:
# measuringlocations = fews.df_in
# measuringlocations

In [ ]:
# waterlevels = fews.df_out
# waterlevels

### Laad en run module (functie) LoadsMatroos voor inlezen belastingen uit Matroos
LET OP: zoek op basis van een measurement_location_code die in (NOOS) Matroos overeenkomt met ['Location' -> loc_id!](https://vitaal.matroos.rws.nl/matroos/php/image_series.php?type=image&source_id=156&loc_id=7&unit_id=1&colors=blue&ymin=0&ymax=0&label=0&alarm=0&alert=0&prealert=0&localtime_offset=0&tcurrent=2024-11-11%2019:38:11&tstart=2024-11-10%2000:00:00&tstop=2024-11-13%2000:00:00&anal_time=2024-11-11%2000:00:00)

In [ ]:
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos = LoadsMatroos(data_adapter=data)
ds = matroos.run(input="BelastingLocaties_Matroos", output="Waterstanden_Matroos")

In [ ]:
measuringlocations = matroos.df_in
measuringlocations

In [ ]:
waterlevels = matroos.df_out
waterlevels

### Laad en run module (functie) LoadsWaterwebservicesRWS voor inlezen belastingen uit RWS Waterwebservices
LET OP: zoek op basis van een measurement_location_code die in RWS WaterWebservices overeenkomt met Locatie_MessageID!\
![image info](data_demo/images/WaterWebservicesRWS_Locatie_MessageId.PNG)

In [ ]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices_rws = LoadsWaterwebservicesRWS(data_adapter=data)
ds = waterwebservices_rws.run(
    input="BelastingLocaties_Waterwebservices_RWS",
    output="Waterstanden_Waterwebservices_RWS",
)

In [ ]:
measuringlocations = waterwebservices_rws.df_in
measuringlocations

In [ ]:
waterlevels = waterwebservices_rws.df_out
waterlevels

### Grafische weergave van resultaten in 4 grafieken